# DnD Monster Data Wrangling


## Importation

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

monster_df = pd.read_csv('../data/raw/Monster_Data_RAW.csv')

monster_df.head()

,Unnamed: 0,Monster Name,Size,Type,Alignment,Traits,Damage Resistances,Monster Tags:,Mythic Actions,Reactions,...,Proficiency Bonus,STR,DEX,CON,INT,WIS,CHA,Actions,Legendary Actions,Environment:
0,0,Adult Green Dragon,Huge,['dragon'],lawful evil,['Amphibious. The dragon can breathe air and w...,NaN,NaN,NaN,NaN,...,5,23,12,21,18,15,17,['Multiattack. The dragon can use its Frightfu...,"[""The dragon can take 3 legendary actions, cho...",['Forest']
1,1,Adult Silver Dragon,Huge,['dragon'],lawful good,['Legendary Resistance (3/Day). If the dragon ...,NaN,NaN,NaN,NaN,...,5,27,10,25,16,13,21,['Multiattack. The dragon can use its Frightfu...,"[""The dragon can take 3 legendary actions, cho...","['Mountain', 'Urban']"
2,2,Adult White Dragon,Huge,['dragon'],chaotic evil,"[""Ice Walk. The dragon can move across and cli...",NaN,NaN,NaN,NaN,...,5,22,10,22,8,12,12,['Multiattack. The dragon can use its Frightfu...,"[""The dragon can take 3 legendary actions, cho...",['Arctic']
3,3,Air Elemental,Large,['elemental'],neutral,"[""Air Form. The elemental can enter a hostile ...","Lightning, Thunder; Bludgeoning, Piercing, and...",NaN,NaN,NaN,...,3,14,20,14,6,10,6,['Multiattack. The elemental makes two slam at...,NaN,"['Desert', 'Mountain']"
4,4,Ape,Medium,['beast'],unaligned,[nan],NaN,['Misc Creature'],NaN,NaN,...,2,16,14,14,6,12,7,['Multiattack. The ape makes two fist attacks....,NaN,['Forest']


## Background Info on Challenge Rating: 
While the Monster Manual has hundreds of monsters ready to go, some times it isn't enough and you, as the dungeon master need to create your own monsters. However, using the guide provided by 5e wizards, somehow never lines up for myself or other DMs the way a the challenge rating for a monster manual does. 

In the Dungeon Master's guide, a challenge rating (CR) is made up primarly from the Armor Class, hit points, attack bonus, damage output per round, and the save DC. It ends up being an average of these items, when related to a chart. 

Additionally, Resistances and Immunities come into play, adding multipliers to the hit points

As far as average damage per round, the dungeon master's guide and wizards themselves indicates that a combat should last roughly 3 rounds. The DMG suggests averaging 3 rounds of combat to calculate the average damage per round. We will fiddle with that a bit though :D

In terms of party composition (this will matter for some AOE, range or aura based attacked), we will assume a standard party of four: 2 close combat and 2 ranged, Fighter, Rogue, Wizard, and Cleric.

Monster Features also impact the challege rating of a monster either through the armor class, hitpoints, attack bonus, or damage output for purposes of determining the CR.

Spell casting is important if a spell causes the monster to deal more damage than a normal attack routine or if it increases the monster AC or hitpoints.

If a monster can fly and deal damage at range it's armor class goes up by 2 if its below 10

Saving throw bonests 3-4 increase AC by 2. 5 or more increase AC by 4.

## Monster Manual Monsters

Using this, let's take a look at a few example Monsters and see how close the CRs compare

Let's take something from both ends of the data set and one in the middle: a classic ancient red dragon @ a 24 CR, a Manticore @ 3 CR, and something with spells, Mummy Lord @ 15 CR

In [8]:
# set  names as index
monster_df = monster_df.set_index("Monster Name")

#remove unnamed column
monster_df.drop('Unnamed: 0',inplace=True, axis=1)

In [22]:
monster_df.columns

Index(['Size', 'Type', 'Alignment', 'Traits', 'Damage Resistances',
       'Monster Tags:', 'Mythic Actions', 'Reactions', 'Source', 'Armor Class',
       'Hit Points', 'Speed', 'Saving Throws', 'Skills',
       'Damage Vulnerabilities', 'Damage Immunities', 'Condition Immunities',
       'Senses', 'Languages', 'Challenge', 'Proficiency Bonus', 'STR', 'DEX',
       'CON', 'INT', 'WIS', 'CHA', 'Actions', 'Legendary Actions',
       'Environment:'],
      dtype='object')

### Ancient Red Dragon


In [50]:
Ancient_Red_Dragon = monster_df.loc[["Ancient Red Dragon"]]


Ancient_Red_Dragon.columns

#filter down columns to HP,AC,Features, Damage per round, Attack Bonus, Save DC, Saving Throws, Spell Casting
Ancient_Red_Dragon = Ancient_Red_Dragon.filter(items=['Armor Class','Hit Points','Damage Resistances','Traits','Damage Immunities', 'Actions', 'Legendary Actions','Reactions', 'Saving Throws' ])

In [52]:
Ancient_Red_Dragon
# No Damage Resistances or Reactions

print(Ancient_Red_Dragon["Actions"].tolist())
# Attack bonus of 17
#Round 1: Fire Breath hits all 4; 364 damage
#Round 2: 55 damage 
#Round 3: 55 damage

print(Ancient_Red_Dragon["Legendary Actions"].tolist())
# 3 tail attacks for 57 damage

#Average damage per round 215
DPR = (364+57+55+57+55+57)/3

print(Ancient_Red_Dragon["Hit Points"])
#HP of 546

print(Ancient_Red_Dragon["Armor Class"])
#AC of 22

print(Ancient_Red_Dragon["Damage Immunities"].tolist())
#fire immunity

print(Ancient_Red_Dragon["Saving Throws"].tolist())
# Four saving throws

['[\'Multiattack. The dragon can use its Frightful Presence. It then makes three attacks: one with its bite and two with its claws.\', \'Bite. Melee Weapon Attack: +17 to hit, reach 15 ft., one target. Hit: 21 (2d10 + 10) piercing damage plus 14 (4d6) fire damage.\', \'Claw. Melee Weapon Attack: +17 to hit, reach 10 ft., one target. Hit: 17 (2d6 + 10) slashing damage.\', \'Tail. Melee Weapon Attack: +17 to hit, reach 20 ft., one target. Hit: 19 (2d8 + 10) bludgeoning damage.\', "Frightful Presence. Each creature of the dragon\'s choice that is within 120 feet of the dragon and aware of it must succeed on a DC 21 Wisdom saving throw or become frightened for 1 minute. A creature can repeat the saving throw at the end of each of its turns, ending the effect on itself on a success. If a creature\'s saving throw is successful or the effect ends for it, the creature is immune to the dragon\'s Frightful Presence for the next 24 hours.", \'Fire Breath (Recharge 5–6). The dragon exhales fire in

HP: 24 CR
AC: 22, will increase defensive CR
Attack bonus: 17
DPR: 25
Only one immunity, so no bonuses there
Four Saving throws means its AC is raised by 2, so 24, 
Legendary Resistance increase HP by 90 total to 636, which is a CR

Defensive CR: HP is 26, a 24 has an AC of 19, 5 points difference: 28.5 CR
Offensive CR: DPR = 25CR, which has an Attack bonus of 12, 5 difference 27.5 CR

Giving us a final CR of 28. The Monster manuals CR is 24!


### Mummy Lord

In [71]:
Mummy_Lord = monster_df.loc[["Mummy Lord"]]

#filter down columns to HP,AC,Features, Damage per round, Attack Bonus, Save DC, Saving Throws, Spell Casting
Mummy_Lord = Mummy_Lord.filter(items=['Armor Class','Hit Points','Damage Resistances','Traits','Damage Immunities', 'Actions', 'Legendary Actions','Reactions', 'Saving Throws' ])

In [72]:
Mummy_Lord
# No Damage Resistances or Reactions

,Armor Class,Hit Points,Damage Resistances,Traits,Damage Immunities,Actions,Legendary Actions,Reactions,Saving Throws
Monster Name,,,,,,,,,
Mummy Lord,17,97,NaN,['Magic Resistance. The mummy lord has advanta...,"Necrotic, Poison; Bludgeoning, Piercing, and S...",['Multiattack. The mummy can use its Dreadful ...,"[""The mummy lord can take 3 legendary actions,...",NaN,"CON +8, INT +5, WIS +9, CHA +8"


In [74]:
print(Mummy_Lord["Actions"].tolist())
# Attack bonus of 9+
#Round 1: Harm @ 49
#Round 2: Insect Plague Radius : 88
#Round 3: 28 damage

print(Mummy_Lord["Legendary Actions"].tolist())
# 3 rotting fist  for 42 damage

#Average damage per round 97
DPR = (49+42+88+42+28+42)/3

print(Mummy_Lord["Hit Points"])
#HP of 97

print(Mummy_Lord["Armor Class"])
#AC of 17

print(Mummy_Lord["Damage Immunities"].tolist())
#over 3 immunities

print(Mummy_Lord["Saving Throws"].tolist())
# Four saving throws

print(Mummy_Lord["Traits"].tolist())
# Magic Resistance


['[\'Multiattack. The mummy can use its Dreadful Glare and makes one attack with its rotting fist.\', "Rotting Fist. Melee Weapon Attack: +9 to hit, reach 5 ft., one target. Hit: 14 (3d6 + 4) bludgeoning damage plus 21 (6d6) necrotic damage. If the target is a creature, it must succeed on a DC 16 Constitution saving throw or be cursed with mummy rot. The cursed target can\'t regain hit points, and its hit point maximum decreases by 10 (3d6) for every 24 hours that elapse. If the curse reduces the target\'s hit point maximum to 0, the target dies, and its body turns to dust. The curse lasts until removed by the remove curse spell or other magic.", "Dreadful Glare. The mummy lord targets one creature it can see within 60 feet of it. If the target can see the mummy lord, it must succeed on a DC 16 Wisdom saving throw against this magic or become frightened until the end of the mummy\'s next turn. If the target fails the saving throw by 5 or more, it is also paralyzed for the same duration

HP: 2 CR
AC: 17, will increase defensive CR
Attack bonus: 9

Save DC: 17
DPR: 18 CR
 over 3 immunity, so multiply HP by 1.5 x bringing it to 145.5
Four Saving throws means its AC is raised by 2, so 19, 
Magic resistance increases AC by 2, so 21

Defensive CR: HP is 6 CR, a 2 has an AC of 13, 8 points difference: 10 CR
Offensive CR: DPR = 15 CR, which has an Save DC of 18, -1 difference 14.5 CR

Giving us a final CR of 12. The Monster manuals CR is 15!


### Manticore

In [75]:
Manitcore = monster_df.loc[["Mummy Lord"]]

#filter down columns to HP,AC,Features, Damage per round, Attack Bonus, Save DC, Saving Throws, Spell Casting
Manitcore = Manitcore.filter(items=['Armor Class','Hit Points','Damage Resistances','Traits','Damage Immunities', 'Actions', 'Legendary Actions','Reactions', 'Saving Throws' ])

In [76]:
Manitcore
# no resistances or reactions


,Armor Class,Hit Points,Damage Resistances,Traits,Damage Immunities,Actions,Legendary Actions,Reactions,Saving Throws
Monster Name,,,,,,,,,
Mummy Lord,17,97,NaN,['Magic Resistance. The mummy lord has advanta...,"Necrotic, Poison; Bludgeoning, Piercing, and S...",['Multiattack. The mummy can use its Dreadful ...,"[""The mummy lord can take 3 legendary actions,...",NaN,"CON +8, INT +5, WIS +9, CHA +8"


In [77]:
print(Manitcore["Actions"].tolist())
# Attack bonus of 9+
#Round 1: Harm @ 49
#Round 2: Insect Plague Radius : 88
#Round 3: 28 damage

print(Manitcore["Legendary Actions"].tolist())
# 3 rotting fist  for 42 damage

#Average damage per round 97
DPR = (49+42+88+42+28+42)/3

print(Manitcore["Hit Points"])
#HP of 97

print(Manitcore["Armor Class"])
#AC of 17

print(Manitcore["Damage Immunities"].tolist())
#over 3 immunities

print(Manitcore["Saving Throws"].tolist())
# Four saving throws

print(Manitcore["Traits"].tolist())
# Magic Resistance


['[\'Multiattack. The mummy can use its Dreadful Glare and makes one attack with its rotting fist.\', "Rotting Fist. Melee Weapon Attack: +9 to hit, reach 5 ft., one target. Hit: 14 (3d6 + 4) bludgeoning damage plus 21 (6d6) necrotic damage. If the target is a creature, it must succeed on a DC 16 Constitution saving throw or be cursed with mummy rot. The cursed target can\'t regain hit points, and its hit point maximum decreases by 10 (3d6) for every 24 hours that elapse. If the curse reduces the target\'s hit point maximum to 0, the target dies, and its body turns to dust. The curse lasts until removed by the remove curse spell or other magic.", "Dreadful Glare. The mummy lord targets one creature it can see within 60 feet of it. If the target can see the mummy lord, it must succeed on a DC 16 Wisdom saving throw against this magic or become frightened until the end of the mummy\'s next turn. If the target fails the saving throw by 5 or more, it is also paralyzed for the same duration